# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [1]:
#Import your libraries
import numpy as np
import pandas as pd
import regex as re

# Introduction

In this lab, we will use two datasets. Both datasets contain variables that describe apps from the Google Play Store. We will use our knowledge in feature extraction to process these datasets and prepare them for the use of a ML algorithm.

# Challenge 1 - Loading and Extracting Features from the First Dataset

#### In this challenge, our goals are: 

* Exploring the dataset.
* Identify the columns with missing values.
* Either replacing the missing values in each column or drop the columns.
* Conver each column to the appropriate type.

#### The first dataset contains different information describing the apps. 

Load the dataset into the variable `google_play` in the cell below. The dataset is in the file `googleplaystore.csv`

In [2]:
# Your code here:
# import libraries 
import numpy as np
import pandas as pandas
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# load dataset into a pandas dataframe
df = pandas.read_csv('googleplaystore.csv')
df.head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


#### Examine all variables and their types in the following cell

In [3]:
# Your code here:
df.shape

(10841, 13)

In [4]:
# assess column names in each data set to see which column is missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [5]:
# assess df's null values
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [6]:
# assess df's duplicated values
sum(df.duplicated())

483

In [7]:
# assess data types of the columns
df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [8]:
# function to make column names lowercase
cols = []
for i in range(len(df.columns)):
    cols.append(df.columns[i].lower())
cols

['app',
 'category',
 'rating',
 'reviews',
 'size',
 'installs',
 'type',
 'price',
 'content rating',
 'genres',
 'last updated',
 'current ver',
 'android ver']

In [9]:
#assign it to columns
df.columns = cols

#### Since this dataset only contains one numeric column, let's skip the `describe()` function and look at the first 5 rows using the `head()` function

In [10]:
# Your code here:
df.head(5)

,app,category,rating,reviews,size,installs,type,price,content rating,genres,last updated,current ver,android ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


#### We can see that there are a few columns that could be coerced to numeric.

Start with the reviews column. We can evaluate what value is causing this column to be of object type finding the non-numeric values in this column. To do this, we recall the `to_numeric()` function. With this function, we are able to coerce all non-numeric data to null. We can then use the `isnull()` function to subset our dataframe using the True/False column that this function generates.

In the cell below, transform the Reviews column to numeric and assign this new column to the variable `Reviews_numeric`. Make sure to coerce the errors.

In [11]:
# Your code here:
df['reviews_numeric'] = pd.to_numeric(df['reviews'], errors='coerce')

In [12]:
df.head()

,app,category,rating,reviews,size,installs,type,price,content rating,genres,last updated,current ver,android ver,reviews_numeric
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,159.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,967.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,87510.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,215644.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,967.0


Next, create a column containing True/False values using the `isnull()` function. Assign this column to the `Reviews_isnull` variable.

In [13]:
# Your code here:
df['reviews_isnull'] = df.isnull().values.any()

In [14]:
df.head()

,app,category,rating,reviews,size,installs,type,price,content rating,genres,last updated,current ver,android ver,reviews_numeric,reviews_isnull
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,159.0,True
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,967.0,True
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,87510.0,True
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,215644.0,True
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,967.0,True


In [15]:
df_reviews_isnull = df['reviews_isnull']

Finally, subset the `google_play` with `Reviews_isnull`. This should give you all the rows that contain non-numeric characters.

Your output should look like:

![Reviews_bool.png](../images/reviews-bool.png)

In [16]:
# Your code here:
df_reviews_isnull.value_counts()

True    10841
Name: reviews_isnull, dtype: int64

#### We see that Google Play is using a shorthand for millions. 

Let's write a function to transform this data.

Steps:

1. Create a function that returns the correct numeric values of *Reviews*.
1. Define a test string with `M` in the last character.
1. Test your function with the test string. Make sure your function works correctly. If not, modify your functions and test again.

In [17]:
# Your code here

def convert_string_to_numeric(s):
    factor = 1
    if s[-1]=='M':
        factor = 1e6
        
    n_df = re.findall('[0-9.]+', s)
    
    return float (n_df[0])*factor

test_string = '4.0M'

convert_string_to_numeric(test_string) == 4000000

True

The last step is to apply the function to the `Reviews` column in the following cell:

In [18]:
# Your code here:
df['reviews'] = df['reviews'].apply(convert_string_to_numeric)

Check the non-numeric `Reviews` row again. It should have been fixed now and you should see:

![Reviews_bool_fixed.png](../images/reviews-bool-fixed.png)

In [19]:
# Your code here
df['reviews_isnull'] = df.isnull().values.any()

In [20]:
df['reviews_isnull'].value_counts()

True    10841
Name: reviews_isnull, dtype: int64

Also check the variable types of `google_play`. The `Reviews` column should be a `float64` type now.

In [21]:
# Your code here:
df.dtypes

app                 object
category            object
rating             float64
reviews            float64
size                object
installs            object
type                object
price               object
content rating      object
genres              object
last updated        object
current ver         object
android ver         object
reviews_numeric    float64
reviews_isnull        bool
dtype: object

#### The next column we will look at is `Size`. We start by looking at all unique values in `Size`:

*Hint: use `unique()` ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html))*.

In [22]:
# Your code here:
df['size'].nunique()

462

In [23]:
df['size'].unique()[:30]

array(['19M', '14M', '8.7M', '25M', '2.8M', '5.6M', '29M', '33M', '3.1M',
       '28M', '12M', '20M', '21M', '37M', '2.7M', '5.5M', '17M', '39M',
       '31M', '4.2M', '7.0M', '23M', '6.0M', '6.1M', '4.6M', '9.2M',
       '5.2M', '11M', '24M', 'Varies with device'], dtype=object)

You should have seen lots of unique values of the app sizes.

#### While we can convert most of the `Size` values to numeric in the same way we converted the `Reviews` values, there is one value that is impossible to convert.

What is that badass value? Enter it in the next cell and calculate the proportion of its occurence to the total number of records of `google_play`.

In [24]:
# Your code here:
(df['size'] == 'Varies with device').sum()

1695

#### While this column may be useful for other types of analysis, we opt to drop it from our dataset. 

There are two reasons. First, the majority of the data are ordinal but a sizeable proportion are missing because we cannot convert them to numerical values. Ordinal data are both numerical and categorical, and they usually can be ranked (e.g. 82k is smaller than 91M). In contrast, non-ordinal categorical data such as blood type and eye color cannot be ranked. The second reason is as a categorical column, it has too many unique values to produce meaningful insights. Therefore, in our case the simplest strategy would be to drop the column.

Drop the column in the cell below (use `inplace=True`)

In [25]:
# Your code here:
df = df.drop(['size'], axis=1)

#### Now let's look at how many missing values are in each column. 

This will give us an idea of whether we should come up with a missing data strategy or give up on the column all together. In the next column, find the number of missing values in each column: 

*Hint: use the `isna()` and `sum()` functions.*

In [26]:
# assess df's null values
df.isnull().sum()

app                   0
category              0
rating             1474
reviews               0
installs              0
type                  1
price                 0
content rating        1
genres                0
last updated          0
current ver           8
android ver           3
reviews_numeric       1
reviews_isnull        0
dtype: int64

You should find the column with the most missing values is now `Rating`.

#### What is the proportion of the missing values in `Rating` to the total number of records?

Enter your answer in the cell below.

In [27]:
# Your code here:
# assess percentage of null values
dfnulls = pd.DataFrame(round(df.isna().sum()/len(df),4)*100)
dfnulls

,0
app,0.00
category,0.00
rating,13.60
reviews,0.00
installs,0.00
type,0.01
price,0.00
content rating,0.01
genres,0.00
last updated,0.00


A sizeable proportion of the `Rating` column is missing. A few other columns also contain several missing values.

#### We opt to preserve these columns and remove the rows containing missing data.

In particular, we don't want to drop the `Rating` column because:

* It is one of the most important columns in our dataset. 

* Since the dataset is not a time series, the loss of these rows will not have a negative impact on our ability to analyze the data. It will, however, cause us to lose some meaningful observations. But the loss is limited compared to the gain we receive by preserving these columns.

In the cell below, remove all rows containing at least one missing value. Use the `dropna()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html)). Assign the new dataframe to the variable `google_missing_removed`.

In [28]:
# Your code here:
google_missing_removed = df.dropna()

In [29]:
google_missing_removed.shape

(9360, 14)

From now on, we use the `google_missing_removed` variable instead of `google_play`.

#### Next, we look at the `Last Updated` column.

The `Last Updated` column seems to contain a date, though it is classified as an object type. Let's convert this column using the `pd.to_datetime` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)).

In [30]:
# Your code here:
google_missing_removed['last updated'] = pd.to_datetime(google_missing_removed['last updated'], format='%B %d %Y', errors = 'coerce')


<ipython-input-30-26ef4dbcd0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  google_missing_removed['last updated'] = pd.to_datetime(google_missing_removed['last updated'], format='%B %d %Y', errors = 'coerce')


#### The last column we will transform is `Price`. 

We start by looking at the unique values of this column.

In [31]:
# Your code here:
google_missing_removed['price'].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$7.99', '$5.99', '$2.99', '$3.49',
       '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49', '$10.00',
       '$24.99', '$11.99', '$79.99', '$16.99', '$14.99', '$29.99',
       '$12.99', '$2.49', '$10.99', '$1.50', '$19.99', '$15.99', '$33.99',
       '$39.99', '$3.95', '$4.49', '$1.70', '$8.99', '$1.49', '$3.88',
       '$399.99', '$17.99', '$400.00', '$3.02', '$1.76', '$4.84', '$4.77',
       '$1.61', '$2.50', '$1.59', '$6.49', '$1.29', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$8.49', '$1.75', '$14.00', '$2.00',
       '$3.08', '$2.59', '$19.40', '$3.90', '$4.59', '$15.46', '$3.04',
       '$13.99', '$4.29', '$3.28', '$4.60', '$1.00', '$2.95', '$2.90',
       '$1.97', '$2.56', '$1.20'], dtype=object)

Since all prices are ordinal data without exceptions, we can tranform this column by removing the dollar sign and converting to numeric. We can create a new column called `Price Numerical` and drop the original column.

We will achieve our goal in three steps. Follow the instructions of each step below.

#### First we remove the dollar sign. Do this in the next cell by applying the `str.replace` function to the column to replace `$` with an empty string (`''`).

In [32]:
# Your code here:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)
google_missing_removed['price'] = google_missing_removed['price'].apply(clean_currency)

<ipython-input-32-582038ac3292>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  google_missing_removed['price'] = google_missing_removed['price'].apply(clean_currency)


#### Second step, coerce the `Price Numerical` column to numeric.

In [33]:
# Your code here:
google_missing_removed['price_numeric'] = pd.to_numeric(google_missing_removed['price'], errors='coerce')

<ipython-input-33-1e5fca18164b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  google_missing_removed['price_numeric'] = pd.to_numeric(google_missing_removed['price'], errors='coerce')


**Finally, drop the original `Price` column.**

In [34]:
# Your code here:
google_missing_removed = google_missing_removed.drop(['price'], axis=1)

Now check the variable types of `google_missing_removed`. Make sure:

* `Size` and `Price` columns have been removed.
* `Rating`, `Reviews`, and `Price Numerical` have the type of `float64`.
* `Last Updated` has the type of `datetime64`.

In [35]:
# Your code here
google_missing_removed.dtypes

app                        object
category                   object
rating                    float64
reviews                   float64
installs                   object
type                       object
content rating             object
genres                     object
last updated       datetime64[ns]
current ver                object
android ver                object
reviews_numeric           float64
reviews_isnull               bool
price_numeric             float64
dtype: object

In [36]:
# remove reviews_isnull column as we don't need that 
google_missing_removed = google_missing_removed.drop(['reviews_isnull'], axis=1)
google_missing_removed = google_missing_removed.drop(['reviews_numeric'], axis=1)

In [37]:
google_missing_removed.dtypes

app                       object
category                  object
rating                   float64
reviews                  float64
installs                  object
type                      object
content rating            object
genres                    object
last updated      datetime64[ns]
current ver               object
android ver               object
price_numeric            float64
dtype: object

In [38]:
google_missing_removed.head()

,app,category,rating,reviews,installs,type,content rating,genres,last updated,current ver,android ver,price_numeric
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159.0,"10,000+",Free,Everyone,Art & Design,NaT,1.0.0,4.0.3 and up,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967.0,"500,000+",Free,Everyone,Art & Design;Pretend Play,NaT,2.0.0,4.0.3 and up,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510.0,"5,000,000+",Free,Everyone,Art & Design,NaT,1.2.4,4.0.3 and up,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644.0,"50,000,000+",Free,Teen,Art & Design,NaT,Varies with device,4.2 and up,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967.0,"100,000+",Free,Everyone,Art & Design;Creativity,NaT,1.1,4.4 and up,0.0


In [39]:
google_missing_removed['price_numeric'].value_counts()

0.00     8715
2.99      114
0.99      106
4.99       70
1.99       59
         ... 
1.61        1
4.60        1
2.90        1
39.99       1
14.00       1
Name: price_numeric, Length: 73, dtype: int64

# Challenge 2 - Loading and Extracting Features from the Second Dataset

Load the second dataset to the variable `google_reviews`. The data is in the file `googleplaystore_user_reviews.csv`.

In [40]:
# Your code here:
google_reviews = pandas.read_csv('googleplaystore_user_reviews.csv')
google_reviews.head()


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


#### This dataset contains the top 100 reviews for each app. 

Let's examine this dataset using the `head` function

In [41]:
# Your code here
google_reviews.head(100)


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000
...,...,...,...,...,...
95,10 Best Foods for You,Boring. I thought actually just texts that's i...,Negative,-0.325000,0.475000
96,10 Best Foods for You,Love app. It gives way good idea I buying stor...,Positive,0.400000,0.400000
97,10 Best Foods for You,Great Its really best unique provides detailed...,Positive,0.579167,0.683333
98,10 Best Foods for You,Nothing special! Could find anything useful!,Positive,0.410714,0.285714


#### The main piece of information we would like to extract from this dataset is the proportion of positive reviews of each app. 

Columns like `Sentiment_Polarity` and `Sentiment_Subjectivity` are not to our interests because we have no clue how to use them. We do not care about `Translated_Review` because natural language processing is too complex for us at present (in fact the `Sentiment`, `Sentiment_Polarity`, and `Sentiment_Subjectivity` columns are derived from `Translated_Review` the data scientists). 

What we care about in this challenge is `Sentiment`. To be more precise, we care about **what is the proportion of *Positive* sentiment of each app**. This will require us to aggregate the `Sentiment` data by `App` in order to calculate the proportions.

Now that you are clear about what we are trying to achieve, follow the steps below that will walk you through towards our goal.

#### Our first step will be to remove all rows with missing sentiment. 

In the next cell, drop all rows with missing data using the `dropna()` function and assign this new dataframe to `review_missing_removed`.

In [42]:
# Your code here:
google_reviews = google_reviews.dropna()


#### Now, use the `value_counts()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html)) to get a sense on how many apps are in this dataset and their review counts.

In [43]:
# Your code here:
google_reviews.value_counts()


App                                                 Translated_Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Sentiment  Sentiment_Polarity  Sentiment_Subjectivity
FastMeet: Chat, Dating, Love                        Good                                                                                                                                                                                                                                                                                                                                                                           

#### Now the tough part comes. Let's plan how we will achieve our goal:

1. We will count the number of reviews that contain *Positive* in the `Sentiment` column.

1. We will create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app.

1. We will then loop the new dataframe to calculate the postivie review portion of each app.

#### Step 1: Count the number of positive reviews.

In the following cell, write a function that takes a column and returns the number of times *Positive* appears in the column. 

*Hint: One option is to use the `np.where()` function ([documentation](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html)).*

In [52]:
# Your code below

def positive_function(x):
    return sum(np.where((x == 'Positive'), True, False))


In [56]:
google_reviews['Sentiment_Positive'] = google_reviews['Sentiment'].apply(positive_function)

TypeError: iteration over a 0-d array

In [45]:
google_reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Positive
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,[1]
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,[1]
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,[1]
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,[1]
5,10 Best Foods for You,Best way,Positive,1.00,0.300000,[1]


In [46]:
google_reviews['Sentiment'].value_counts()

Positive    23998
Negative     8271
Neutral      5158
Name: Sentiment, dtype: int64

In [47]:
google_reviews['Sentiment_Positive'].value_counts()

TypeError: unhashable type: 'numpy.ndarray'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'numpy.ndarray'


[1]    23998
[]         1
[]         1
[]         1
[]         1
       ...  
[]         1
[]         1
[]         1
[]         1
[]         1
Name: Sentiment_Positive, Length: 13430, dtype: int64

#### Step 2: Create a new dataframe to contain the `App` name, the number of positive reviews, and the total number of reviews of each app

We will group `review_missing_removed` by the `App` column, then aggregate the grouped dataframe on the number of positive reviews and the total review counts of each app. The result will be assigned to a new variable `google_agg`. Here is the ([documentation on how to achieve it](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)). Take a moment or two to read the documentation and google examples because it is pretty complex.

When you obtain `google_agg`, check its values to make sure it has an `App` column as its index as well as a `Positive` column and a `Total` column. Your output should look like:

![Positive Reviews Agg](../images/positive-review-agg.png)

*Hint: Use `positive_function` you created earlier as part of the param passed to the `agg()` function in order to aggregate the number of positive reviews.*

#### Bonus:

As of Pandas v0.23.4, you may opt to supply an array or an object to `agg()`. If you use the array param, you'll need to rename the columns so that their names are `Positive` and `Total`. Using the object param will allow you to create the aggregated columns with the desirable names without renaming them. However, you will probably encounter a warning indicating supplying an object to `agg()` will become outdated. It's up to you which way you will use. Try both ways out. Any way is fine as long as it works.

In [54]:
# Your code here:
google_agg = google_reviews.groupby(['App']).agg({'Sentiment': [positive_function, 'count']})

Print the first 5 rows of `google_agg` to check it.

In [55]:
# Your code here
google_agg.head(5)

Sentiment      
                                 positive_function count
App                                                     
10 Best Foods for You                          162   194
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                31    40
11st                                            23    39
1800 Contacts - Lens Store                      64    80
1LINE – One Line with One Touch                 27    38

#### Add a derived column to `google_agg` that is the ratio of the `Positive` and the `Total` columns. Call this column `Positive Ratio`. 

Make sure to account for the case where the denominator is zero using the `np.where()` function.

In [ ]:
# Your code here:
google_agg['Positive ratio'] = google_agg['Positive'] / google_agg['Total']

#### Now drop the `Positive` and `Total` columns. Do this with `inplace=True`.

In [ ]:
# Your code here:
google_agg = google_agg.drop(['Positive', 'Total', axis=1)

Print the first 5 rows of `google_agg`. Your output should look like:

![Positive Reviews Agg](../images/positive-review-ratio.png)

In [ ]:
# Your code here:
google_agg.head(5)

# Challenge 3 - Join the Dataframes

In this part of the lab, we will join the two dataframes and obtain a dataframe that contains features we can use in our ML algorithm.

In the next cell, join the `google_missing_removed` dataframe with the `google_agg` dataframe on the `App` column. Assign this dataframe to the variable `google`.

In [ ]:
# Your code here:
google = df_missing_removed.merge(google_agg, left_on='App', right_on = 'App')


#### Let's look at the final result using the `head()` function. Your final product should look like:

![Final Product](../images/google-final-head.png)

In [ ]:
# Your code here:
google.head()